In [1]:
from FactorSet import *
from gp_update_function import *
from InitStrategies import *
from PlotWrapper import *
from plan_next_experiment import *
import matplotlib.pyplot as plt

In [2]:
#create a FactorSet class
my_factors = FactorSet()
my_factors.add_categorical(name="projectile", levels=["pingpong", "whiffle"])
my_factors.add_continuous(name="draw_angle", minimum=0, maximum=180)
my_factors.add_ordinal(name="rubber_bands", levels=[1, 2, 3])
my_factors.factors

[['draw_angle', [0, 180], 'Continuous'],
 ['projectile', ['pingpong', 'whiffle'], 'Categorical'],
 ['rubber_bands', [1, 2, 3], 'Ordinal']]

In [16]:
#create all 3 designs: LHS, maximin, random
X_LHS = latin_hypercube_design(my_factors,6)
X_maximin = maximin_design(my_factors,6)
X_random = random_design(my_factors,6)

/Users/noelletoong/Documents/UMich/Research/PyAutoSci/InitStrategies.py:149: UserWarning:

Using discrete factors whose levels are not the same as the number of runs will not result in a ture Latin Hypercube sampling design. The discrete factors with different numbers of levels compared to runs: draw_angle, projectile



In [17]:
#plot all 3
plot_design(X_LHS,my_factors,"Latin Hypercube Sampling design")
plot_design(X_maximin,my_factors,"Maximin design")
plot_design(X_random,my_factors,"Random design")

In [14]:
#create a Gaussian Process
gp = GaussianProcessRegressor()

In [44]:
#Suppose we start with maximin design, train GP on maximin design
#First display X_maximin decoded
X_LHS

,draw_angle,projectile,rubber_bands
0,15.0,pingpong,3.0
1,105.0,pingpong,1.0
2,45.0,whiffle,2.0
3,75.0,whiffle,2.0
4,165.0,whiffle,1.0
5,135.0,pingpong,3.0


In [19]:
#record "responses"
y = [0.5, 1.3, 1.6, 1.7, 1.5, 1.2]

In [46]:
#train the GP on X_maximin and y
gp = train_model(gp,X_LHS,y,my_factors)


Encoded discrete factors:
    draw_angle  projectile  rubber_bands
0        15.0         0.0           2.0
1       105.0         0.0           0.0
2        45.0         1.0           1.0
3        75.0         1.0           1.0
4       165.0         1.0           0.0
5       135.0         0.0           2.0


In [47]:
factor_names = [factor[0] for factor in my_factors.factors]

In [29]:
#want to suggest next experiment, let's say with exploitation
next_experiment_exploitation = plan_next_experiment(y, my_factors, gp, "Exploitation",random_restarts=5)

On random restart number  0
all_possible_improvements_X_continuous
 [array([170.68805015]), array([170.61941239]), array([170.68805015]), array([170.61941239]), array([170.46477768]), array([170.61941239]), array([4.43811691]), array([4.43811691]), array([4.43811691]), array([4.43811691]), array([4.43811691]), array([4.43811691]), array([123.3095849]), array([123.3095849]), array([123.3095849]), array([123.3095849]), array([123.3095849]), array([123.3095849]), array([108.38832024]), array([108.38832024]), array([108.38832024]), array([108.38832024]), array([108.38832024]), array([108.38832024]), array([97.98074235]), array([97.98074235]), array([97.98074235]), array([97.98074235]), array([97.98074235]), array([97.98074235])]


all_possible_improvements_X_discrete
 [array([0, 0]), array([0, 1]), array([0, 2]), array([1, 0]), array([1, 1]), array([1, 2]), array([0, 0]), array([0, 1]), array([0, 2]), array([1, 0]), array([1, 1]), array([1, 2]), array([0, 0]), array([0, 1]), array([0, 2]),

In [30]:
next_experiment_exploitation

(   draw_angle projectile  rubber_bands
 0   97.980742    whiffle           1.0,
 3.013902469911329e-85)

In [31]:
#want to suggest next experiment, let's say with exploration
next_experiment_exploration = plan_next_experiment(y, my_factors, gp, "Exploration",random_restarts=5)

On random restart number  0
all_possible_improvements_X_continuous
 [array([11.9922967]), array([11.9922967]), array([11.9922967]), array([11.9922967]), array([11.9922967]), array([11.9922967]), array([98.29886363]), array([98.29886363]), array([98.29886363]), array([98.29886363]), array([98.29886363]), array([98.29886363]), array([59.91849562]), array([59.91849562]), array([59.91849562]), array([59.91849562]), array([59.91849562]), array([59.91849562]), array([179.10058531]), array([179.21227884]), array([179.10058531]), array([179.21227884]), array([179.42526104]), array([179.21227884]), array([120.66022735]), array([121.13983437]), array([121.24379887]), array([120.08390734]), array([120.92271635]), array([121.13983437])]


all_possible_improvements_X_discrete
 [array([0, 0]), array([0, 1]), array([0, 2]), array([1, 0]), array([1, 1]), array([1, 2]), array([0, 0]), array([0, 1]), array([0, 2]), array([1, 0]), array([1, 1]), array([1, 2]), array([0, 0]), array([0, 1]), array([0, 2]),

In [32]:
next_experiment_exploration

(   draw_angle projectile  rubber_bands
 0  120.083907    whiffle           1.0,
 -0.9999926076378989)

In [42]:
#want to suggest next experiment, let's say with expected improvement
next_experiment_EI = plan_next_experiment(y, my_factors, gp, "EI")

On random restart number  0
all_possible_improvements_X_continuous
 [array([77.47143723]), array([77.47143723]), array([77.47143723]), array([77.47143723]), array([77.47143723]), array([77.47143723]), array([23.12919097]), array([23.4638897]), array([8.59776279]), array([16.47721686]), array([23.78060708]), array([11.54444374])]


all_possible_improvements_X_discrete
 [array([0, 0]), array([0, 1]), array([0, 2]), array([1, 0]), array([1, 1]), array([1, 2]), array([0, 0]), array([0, 1]), array([0, 2]), array([1, 0]), array([1, 1]), array([1, 2])]


all_possible_improvements_y
 [-0.6977965574013061, -0.6977965574013061, -0.6977965574013061, -0.6977965574013061, -0.6977965574013061, -0.6977965574013061, -0.6977951453191287, -0.6977952062638821, -0.697796557401293, -1.5295577955322329e-10, -0.6977960912970838, -0.6977946173772516]


   draw_angle  projectile  rubber_bands
0   16.477217         1.0           0.0


In [43]:
next_experiment_EI

(   draw_angle projectile  rubber_bands
 0   16.477217    whiffle           1.0,
 -1.5295577955322329e-10)